In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [61]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_tx.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,1904 W GRAND PKWY N,KATY,TX,77449.0,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,NaN
1,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,1800 BERING DR STE 800,HOUSTON,TX,77057.0,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,NaN
2,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",2801 NETWORK BLVD SUITE 300,FRISCO,TX,75034.0,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,NaN
3,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",5000 Overton Plaza Suite 300,Fort Worth,TX,76109.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,NaN
4,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",5601 W WACO DR,WACO,TX,76710.0,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,886818,9936567203,4/28/20,671,PPP,ST. ANTHONY CATHOLIC CHURCH,401 South Parker,Bryan,TX,77803.0,...,999 Other,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN
76181,886819,9960748303,1/31/21,681,PPS,URCORE COMPANIES LLC,1386 Industrial Dr Ste 125,New Braunfels,TX,78130.0,...,999 Other,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN
76182,886820,9966447007,4/9/20,678,PPP,STEADFAST ENERGY INVESTMENTS LP,4305 N. Garfield,Midland,TX,79705.0,...,999 Other,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN
76183,886821,9967088603,3/26/21,677,PPP,RUDDOCK MANUFACTURING COMPANY INC.,1801 Magoffin Ave,El Paso,TX,79901.0,...,315 Apparel,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN


In [62]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [63]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
29844,6582497008,"806 PO Box, MELVIN, TX",NaN,NaN
40231,4283458309,"325, DALLAS, TX",NaN,NaN
70645,7955037106,"156 ENCANTADA, HORSESHOE LANE, TX",NaN,NaN
75408,4766927306,"2715 HANCOCK DR, AUSTIN, TX",NaN,NaN


In [64]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,1063697200,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741
1,1268757405,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116
2,1269717210,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479
3,1648657107,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639
4,1657727204,"5601 W WACO DR, WACO, TX",31.521110,-97.184595


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [26]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
fips_check_scraped.head() #15869

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,0,0,0,810638,810638,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
1,1,1,1,1,1,1,1,1,810639,810639,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
2,2,2,2,2,2,2,2,2,810640,810640,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
3,3,3,3,3,3,3,3,3,810641,810641,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14
4,4,4,4,4,4,4,4,4,810642,810642,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14


In [27]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [29]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1063697200,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
1,1268757405,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
2,1269717210,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
3,1648657107,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14
4,1657727204,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14
...,...,...,...,...,...
76180,9936567203,"401 South Parker, Bryan, TX",30.670456,-96.374926,4.804100e+14
76181,9960748303,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,4.809131e+14
76182,9966447007,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,4.832900e+14
76183,9967088603,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,4.814100e+14


In [30]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
25102,6879468301,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
27097,1259317204,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
29216,5917947010,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
29576,4722878309,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
32150,4657767002,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
33564,4627727109,"1401 West US HIGHWAY 77, SAN BENITO, TX",36.299336,-97.283176,NaN
36500,1047477106,"US HIGHWAY 87, ADKINS, TX",38.455217,-104.606642,NaN
37505,3192617705,NaN,36.127071,-79.860718,NaN
39715,7675668403,"2513 Highway 75 S, Madisonville, TX",33.955462,-96.415656,NaN
40819,8780607004,"100 I-35 North, WACO, TX",34.587650,-97.205132,NaN


<h4>FIPS Script

In [7]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [8]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager().install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('48')] #TX
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

[WDM] - Downloading: 100%|██████████| 8.84M/8.84M [00:00<00:00, 9.63MB/s]


  0%|          | 0/22416 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [9]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

22416


Matching FIPS to DataFrame

In [12]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:25062] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [13]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
25102,6879468301,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,[]
27097,1259317204,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,[]
29216,5917947010,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,[]
29576,4722878309,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,[]
32150,4657767002,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,[]
...,...,...,...,...,...
76180,9936567203,"401 South Parker, Bryan, TX",30.670456,-96.374926,[480410006033001]
76181,9960748303,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,[480913105032017]
76182,9966447007,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,[483290003051000]
76183,9967088603,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,[481410021003012]


Remove brackets

In [14]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
25102,6879468301,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
27097,1259317204,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
29216,5917947010,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
29576,4722878309,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
32150,4657767002,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
...,...,...,...,...,...
76180,9936567203,"401 South Parker, Bryan, TX",30.670456,-96.374926,480410006033001
76181,9960748303,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,480913105032017
76182,9966447007,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,483290003051000
76183,9967088603,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,481410021003012


In [15]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
25102,6879468301,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
27097,1259317204,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
29216,5917947010,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
29576,4722878309,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
32150,4657767002,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
33564,4627727109,"1401 West US HIGHWAY 77, SAN BENITO, TX",36.299336,-97.283176,NaN
36500,1047477106,"US HIGHWAY 87, ADKINS, TX",38.455217,-104.606642,NaN
37505,3192617705,NaN,36.127071,-79.860718,NaN
39715,7675668403,"2513 Highway 75 S, Madisonville, TX",33.955462,-96.415656,NaN
40819,8780607004,"100 I-35 North, WACO, TX",34.587650,-97.205132,NaN


Parse for Merge

In [16]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
25102,6879468301,NaN
27097,1259317204,NaN
29216,5917947010,NaN
29576,4722878309,NaN
32150,4657767002,NaN
...,...,...
76180,9936567203,480410006033001
76181,9960748303,480913105032017
76182,9966447007,483290003051000
76183,9967088603,481410021003012


Read in base data

In [17]:
#if from geocode file
#fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_tx.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check.head()

#if from FIPS file
fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,0,0,810638,810638,810638,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
1,1,1,1,1,1,1,1,810639,810639,810639,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
2,2,2,2,2,2,2,2,810640,810640,810640,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
3,3,3,3,3,3,3,3,810641,810641,810641,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14
4,4,4,4,4,4,4,4,810642,810642,810642,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14


Merge with MainFrame

In [18]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,0,0,0,0,0,0,810638,810638,810638,...,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14,NaN
1,1,1,1,1,1,1,1,810639,810639,810639,...,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14,NaN
2,2,2,2,2,2,2,2,810640,810640,810640,...,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14,NaN
3,3,3,3,3,3,3,3,810641,810641,810641,...,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14,NaN
4,4,4,4,4,4,4,4,810642,810642,810642,...,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,76180,76180,76180,76180,76180,76180,76180,886818,886818,886818,...,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN,480410006033001
76181,76181,76181,76181,76181,76181,76181,76181,886819,886819,886819,...,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN,480913105032017
76182,76182,76182,76182,76182,76182,76182,76182,886820,886820,886820,...,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN,483290003051000
76183,76183,76183,76183,76183,76183,76183,76183,886821,886821,886821,...,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN,481410021003012


<b>Fill FIPS Columns - if FIPS file

In [19]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,0,0,0,0,0,810638,810638,810638,...,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14,NaN,482015429014002.0
1,1,1,1,1,1,1,1,810639,810639,810639,...,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14,NaN,482014314042001.0
2,2,2,2,2,2,2,2,810640,810640,810640,...,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14,NaN,480850305041027.0
3,3,3,3,3,3,3,3,810641,810641,810641,...,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14,NaN,484391054055006.0
4,4,4,4,4,4,4,4,810642,810642,810642,...,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14,NaN,483090023022005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,76180,76180,76180,76180,76180,76180,76180,886818,886818,886818,...,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN,480410006033001,480410006033001
76181,76181,76181,76181,76181,76181,76181,76181,886819,886819,886819,...,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN,480913105032017,480913105032017
76182,76182,76182,76182,76182,76182,76182,76182,886820,886820,886820,...,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN,483290003051000,483290003051000
76183,76183,76183,76183,76183,76183,76183,76183,886821,886821,886821,...,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN,481410021003012,481410021003012


Drop Extra FIPS Columns

In [20]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,0,0,810638,810638,810638,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,482015429014002.0
1,1,1,1,1,1,1,1,810639,810639,810639,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,482014314042001.0
2,2,2,2,2,2,2,2,810640,810640,810640,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,480850305041027.0
3,3,3,3,3,3,3,3,810641,810641,810641,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,484391054055006.0
4,4,4,4,4,4,4,4,810642,810642,810642,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,483090023022005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,76180,76180,76180,76180,76180,76180,76180,886818,886818,886818,...,999 Other,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,480410006033001
76181,76181,76181,76181,76181,76181,76181,76181,886819,886819,886819,...,999 Other,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,480913105032017
76182,76182,76182,76182,76182,76182,76182,76182,886820,886820,886820,...,999 Other,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,483290003051000
76183,76183,76183,76183,76183,76183,76183,76183,886821,886821,886821,...,315 Apparel,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,481410021003012


In [21]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
25102,25102,25102,25102,25102,25102,25102,25102,835740,835740,835740,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
27097,27097,27097,27097,27097,27097,27097,27097,837735,837735,837735,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
29216,29216,29216,29216,29216,29216,29216,29216,839854,839854,839854,...,999 Other,4842 Specialized Freight Trucking,4842.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
29576,29576,29576,29576,29576,29576,29576,29576,840214,840214,840214,...,999 Other,4411 Automobile Dealers,4411.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
29844,29844,29844,29844,29844,29844,29844,29844,840482,840482,840482,...,999 Other,4931 Warehousing and Storage,4931.0,SAN ANTONIO DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"806 PO Box, MELVIN, TX",NaN,NaN,NaN
32150,32150,32150,32150,32150,32150,32150,32150,842788,842788,842788,...,999 Other,4244 Grocery and Related Product Merchant Whol...,4244.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
33564,33564,33564,33564,33564,33564,33564,33564,844202,844202,844202,...,999 Other,4411 Automobile Dealers,4411.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1401 West US HIGHWAY 77, SAN BENITO, TX",36.299336,-97.283176,NaN
36500,36500,36500,36500,36500,36500,36500,36500,847138,847138,847138,...,999 Other,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"US HIGHWAY 87, ADKINS, TX",38.455217,-104.606642,NaN
37505,37505,37505,37505,37505,37505,37505,37505,848143,848143,848143,...,999 Other,8132 Grantmaking and Giving Services,8132.0,DALLAS / FT WORTH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,NaN,36.127071,-79.860718,NaN
39715,39715,39715,39715,39715,39715,39715,39715,850353,850353,850353,...,999 Other,2131 Support Activities for Mining,2131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2513 Highway 75 S, Madisonville, TX",33.955462,-96.415656,NaN


Convert to File

In [22]:
fips_TX = fips_merge_drop

In [23]:
fips_TX.to_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 

Review & Compare

In [24]:
geo_TX_fips = pd.read_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
geo_TX_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,0,0,0,810638,810638,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
1,1,1,1,1,1,1,1,1,810639,810639,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
2,2,2,2,2,2,2,2,2,810640,810640,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
3,3,3,3,3,3,3,3,3,810641,810641,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14
4,4,4,4,4,4,4,4,4,810642,810642,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14


In [25]:
##### fill in missing Lat, Long, FIPS
geo_TX_fips.loc[geo_TX_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
25102,25102,25102,25102,25102,25102,25102,25102,25102,835740,835740,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
27097,27097,27097,27097,27097,27097,27097,27097,27097,837735,837735,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
29216,29216,29216,29216,29216,29216,29216,29216,29216,839854,839854,...,999 Other,4842 Specialized Freight Trucking,4842.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
29576,29576,29576,29576,29576,29576,29576,29576,29576,840214,840214,...,999 Other,4411 Automobile Dealers,4411.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
29844,29844,29844,29844,29844,29844,29844,29844,29844,840482,840482,...,999 Other,4931 Warehousing and Storage,4931.0,SAN ANTONIO DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"806 PO Box, MELVIN, TX",NaN,NaN,NaN
32150,32150,32150,32150,32150,32150,32150,32150,32150,842788,842788,...,999 Other,4244 Grocery and Related Product Merchant Whol...,4244.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
33564,33564,33564,33564,33564,33564,33564,33564,33564,844202,844202,...,999 Other,4411 Automobile Dealers,4411.0,LOWER RIO GRANDE VALLEY DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1401 West US HIGHWAY 77, SAN BENITO, TX",36.299336,-97.283176,NaN
36500,36500,36500,36500,36500,36500,36500,36500,36500,847138,847138,...,999 Other,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"US HIGHWAY 87, ADKINS, TX",38.455217,-104.606642,NaN
37505,37505,37505,37505,37505,37505,37505,37505,37505,848143,848143,...,999 Other,8132 Grantmaking and Giving Services,8132.0,DALLAS / FT WORTH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,NaN,36.127071,-79.860718,NaN
39715,39715,39715,39715,39715,39715,39715,39715,39715,850353,850353,...,999 Other,2131 Support Activities for Mining,2131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2513 Highway 75 S, Madisonville, TX",33.955462,-96.415656,NaN


In [39]:
geo_TX_fips["BusinessType"] = geo_TX_fips["BusinessType"].fillna('Not Available')
geo_TX_fips["BusinessType"].unique()
geo_TX_fips.to_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
geo_TX_fips =pd.read_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
geo_TX_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Subchapter S Corporation',
       'Corporation', 'Partnership', 'Limited Liability Partnership',
       '501(c)3 – Non Profit', 'Employee Stock Ownership Plan(ESOP)',
       'Professional Association', 'Sole Proprietorship',
       'Non-Profit Organization', 'Not Available', 'Joint Venture',
       'Cooperative', '501(c)6 – Non Profit Membership',
       'Self-Employed Individuals', 'Trust', 'Independent Contractors',
       'Non-Profit Childcare Center', 'Single Member LLC',
       'Tenant in Common', 'Qualified Joint-Venture (spouses)'],
      dtype=object)